In [1]:
# !pip install -q transformers
# !pip install -q tf-models-official==2.2.0
# !pip install pyvi


In [1]:
import os
import time
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoConfig, AutoModel, BertModel, BertPreTrainedModel, BertConfig, RobertaConfig, RobertaModel, BertTokenizer, BertModel, RobertaTokenizer
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
import torch.nn as nn
from torch.autograd import Function
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from official import nlp
import official.nlp.optimization
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import logging
from tqdm import tqdm, trange
from pyvi.ViTokenizer import tokenize


In [2]:
#Get the GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('GPU: {}'.format(device_name))
else:
  raise SystemError('GPU not found')

GPU: /device:GPU:0


In [3]:
# If there's a GPU
if torch.cuda.is_available():
  device = torch.device('cuda')
  torch.cuda.set_device(0)
  print('There are %d GPU(s).' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name())
else:
  print('No GPU, using CPU.')
  device = torch.device('cpu')

There are 1 GPU(s).
We will use the GPU: NVIDIA GeForce RTX 3090


In [125]:
base_dir    = '.'
train_path  = os.path.join(base_dir, 'fix-train.csv')
val_path    = os.path.join(base_dir, 'fix-val.csv')


In [126]:
df_train = pd.read_csv(train_path)
print(df_train.shape)
print(df_train.info())

display(df_train.head())

(25614, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25614 entries, 0 to 25613
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          25614 non-null  object
 1   context        25614 non-null  object
 2   answer_text    25614 non-null  object
 3   answer_start   25614 non-null  int64 
 4   id             25614 non-null  object
 5   is_impossible  25614 non-null  bool  
 6   question       25614 non-null  object
 7   answer_end     25614 non-null  int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 1.4+ MB
None


,title,context,answer_text,answer_start,id,is_impossible,question,answer_end
0,Pháp,"Trong tháng 8 năm 1791, Hoàng đế Áo và Quốc vư...",nhóm 'Gironde' ủng hộ chiến tranh với Áo và Ph...,475,uit_024488,False,Hội nghị Lập pháp ở Pháp đã tồn tại hai phe đố...,568
1,Lịch sử Hoa Kỳ,Phe bảo hoàng mà người Anh trông cậy quá nhiều...,tháng 11 năm 1783,249,uit_017793,True,Đội quân cuối cùng của New Yord đã rời khỏi An...,266
2,Trung Hoa Dân Quốc (1912-1949),"Trong lĩnh vực toán học, Trung Hoa Dân Quốc đạ...",tiên phong và khai sáng vật lý học cận đại Tru...,177,uit_019463,False,Điều gì đã được làm bởi Ngô Hữu Huấn?,230
3,Iran,Iran là một thành viên sáng lập của Liên Hiệp ...,22 di sản,488,uit_022453,False,Có bao nhiêu di sản ở Iran được UNESCO công nhận?,497
4,Cách mạng Tháng Mười,"Ngày 10-1-1918, Đại hội Xô viết toàn Nga lần t...",Xô viết đại biểu nông dân với Xô viết đại biểu...,98,uit_011191,True,Đại hội đã quyết định thành lập hai Xô viết đạ...,167


In [127]:
df_val = pd.read_csv(val_path)
print(df_val.shape)
print(df_val.info())

display(df_val.head())

(2846, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2846 entries, 0 to 2845
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2846 non-null   object
 1   context        2846 non-null   object
 2   answer_text    2846 non-null   object
 3   answer_start   2846 non-null   int64 
 4   id             2846 non-null   object
 5   is_impossible  2846 non-null   bool  
 6   question       2846 non-null   object
 7   answer_end     2846 non-null   int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 158.5+ KB
None


,title,context,answer_text,answer_start,id,is_impossible,question,answer_end
0,Donald Trump,Một phần nhỏ tài sản của Trump nằm trong các k...,quyết định thâm nhập thị trường cổ phiếu,201,uit_015587,False,Hành động gì của Trump đã gây ra bất ngờ đối v...,241
1,Michael Jackson,Chuyến lưu diễn HIStory World Tour bắt đầu từ ...,"82 đêm nhạc tại 58 thành phố, đi qua 5 châu lụ...",152,uit_008735,False,Chặng đường của HIStory World Tour như thế nào?,250
2,Bức tường Berlin,Công dân Đông Đức đã được người dân Tây Berlin...,tặng 100 DM khi qua cổng,458,uit_021854,False,Chính phủ chào đón người dân Đông Đức như thế ...,482
3,Nhà Minh,"Sau khi Minh Thành Tổ lên ngôi, trong những nă...",triều đình nhận thấy tính quan trọng của mậu d...,1261,uit_018747,False,Vì sao triều đình lại từng bước xóa bỏ lệnh hả...,1344
4,Đế quốc La Mã,"Cuộc chiến tranh Do Thái-La Mã lần thứ nhất, đ...",Cả hai cuộc khởi nghĩa này đều bị đàn áp dã man,800,uit_020519,False,Kết quả của hai cuộc khởi nghĩa của người Do T...,847


In [138]:
df_train.iloc[0]

title                                                         Pháp
context          Trong tháng 8 năm 1791, Hoàng đế Áo và Quốc vư...
answer_text      nhóm 'Gironde' ủng hộ chiến tranh với Áo và Ph...
answer_start                                                   475
id                                                      uit_024488
is_impossible                                                False
question         Hội nghị Lập pháp ở Pháp đã tồn tại hai phe đố...
answer_end                                                     568
Name: 0, dtype: object

In [ ]:
import re

train_q  = df_train.question.values
train_a  = df_train.answer_text.values
train_c  = df_train.context.values
train_ans_start = df_train.answer_start.values
train_ans_end = df_train.answer_end.values

val_q  = df_train.question.values
val_a  = df_train.answer_text.values
val_c  = df_train.context.values
val_ans_start = df_train.answer_start.values
val_ans_end = df_train.answer_end.values

# test_sent    = df_val.tokenized.values
# test_target = df_val.Target.values
# test_stance_labels   = df_val.enc_label.values

In [ ]:
MODEL       = '/content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/checkpoint-810000'
MAX_LENGTH    = 500
MAX_LENGTH_Q  = 70
# MAX_LENGTH_Q  = 256

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained(MODEL)

Loading BERT tokenizer...


In [ ]:
input_ids_1 = []
attention_masks_1 = []
token_type_ids_1 = []
input_ids_2 = []
attention_masks_2 = []
token_type_ids_2 = []

for sent1, sent2 in zip(train_sent_1, train_sent_2):
  encoded_dict = tokenizer.encode_plus( sent1, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH_Q,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_1.append(encoded_dict['input_ids'])
  attention_masks_1.append(encoded_dict['attention_mask'])
  token_type_ids_1.append(encoded_dict['token_type_ids'])

  encoded_dict = tokenizer.encode_plus( sent2, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_2.append(encoded_dict['input_ids'])
  attention_masks_2.append(encoded_dict['attention_mask'])
  token_type_ids_2.append(encoded_dict['token_type_ids'])


train_input_ids_1 = torch.tensor(input_ids_1)
train_attention_masks_1 = torch.tensor(attention_masks_1)
train_token_type_ids_1 = torch.tensor(token_type_ids_1)
train_input_ids_2 = torch.tensor(input_ids_2)
train_attention_masks_2 = torch.tensor(attention_masks_2)
train_token_type_ids_2 = torch.tensor(token_type_ids_2)
train_labels = torch.tensor(train_labels)
train_domains = torch.tensor(train_domains)
train_input_ids_1.size(), train_attention_masks_1.size(), train_token_type_ids_1.size(), train_input_ids_2.size(), train_attention_masks_2.size(), train_token_type_ids_2.size(), train_labels.size()

(torch.Size([3308, 70]),
 torch.Size([3308, 70]),
 torch.Size([3308, 70]),
 torch.Size([3308, 500]),
 torch.Size([3308, 500]),
 torch.Size([3308, 500]),
 torch.Size([3308]))

In [ ]:
input_ids_1 = []
attention_masks_1 = []
token_type_ids_1 = []
input_ids_2 = []
attention_masks_2 = []
token_type_ids_2 = []

for sent1, sent2 in zip(val_sent_1, val_sent_2):
  encoded_dict = tokenizer.encode_plus( sent1, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH_Q,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_1.append(encoded_dict['input_ids'])
  attention_masks_1.append(encoded_dict['attention_mask'])
  token_type_ids_1.append(encoded_dict['token_type_ids'])

  encoded_dict = tokenizer.encode_plus( sent2, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_2.append(encoded_dict['input_ids'])
  attention_masks_2.append(encoded_dict['attention_mask'])
  token_type_ids_2.append(encoded_dict['token_type_ids'])


val_input_ids_1 = torch.tensor(input_ids_1)
val_attention_masks_1 = torch.tensor(attention_masks_1)
val_token_type_ids_1 = torch.tensor(token_type_ids_1)
val_input_ids_2 = torch.tensor(input_ids_2)
val_attention_masks_2 = torch.tensor(attention_masks_2)
val_token_type_ids_2 = torch.tensor(token_type_ids_2)
val_labels = torch.tensor(val_labels)
val_domains = torch.tensor(val_domains)
val_input_ids_1.size(), val_attention_masks_1.size(), val_token_type_ids_1.size(), val_input_ids_2.size(), val_attention_masks_2.size(), val_token_type_ids_2.size(), val_labels.size()

(torch.Size([832, 70]),
 torch.Size([832, 70]),
 torch.Size([832, 70]),
 torch.Size([832, 500]),
 torch.Size([832, 500]),
 torch.Size([832, 500]),
 torch.Size([832]))

In [ ]:
train_data = TensorDataset(train_input_ids_1, train_attention_masks_1, train_token_type_ids_1, train_input_ids_2, train_attention_masks_2, train_token_type_ids_2, train_labels, train_domains)
val_data = TensorDataset(val_input_ids_1, val_attention_masks_1, val_token_type_ids_1, val_input_ids_2, val_attention_masks_2, val_token_type_ids_2, val_labels, val_domains)
# test_data = TensorDataset(test_input_ids, test_attention_masks, test_token_type_ids, test_atheism, test_abortion, test_climate, test_feminist, test_stance_labels)


In [ ]:
class ReverseLayer(Function):
  @staticmethod
  def forward(ctx, x, alpha):
    ctx.alpha = alpha

    return x.view_as(x) 
  
  @staticmethod
  def backward(ctx, grad_out):
    output = grad_out.neg() * ctx.alpha
    
    return output, None

class GRL(torch.nn.Module):
    def __init__(self, lambda_=1):
        super(GRL, self).__init__()
        self.lambda_ = lambda_

    def forward(self, x):
        return ReverseLayer.apply(x, self.lambda_)


In [ ]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.0, use_activation='None', set_bias=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim, bias=set_bias)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        self.dropout_rate = dropout_rate

    def forward(self, x):
        if self.dropout_rate!=0.0:
          temp = self.dropout(x)
        else:
          temp = x

        if self.use_activation=='relu':
            return self.relu(self.linear(temp))
        if self.use_activation=='tanh':
            return self.tanh(self.linear(temp))
        if self.use_activation=='softmax':
            return self.softmax(self.linear(temp))
        if self.use_activation=='sigmoid':
            return self.sigmoid(self.linear(temp))

        return self.linear(temp)


In [ ]:
class CNN_Feature(nn.Module):
  def __init__(self, hidden_size, debug=0):
    super(CNN_Feature, self).__init__()

    filter_sizes = [2,3,4,5]
    batch_norm = []
    self.conv_layers = nn.ModuleList()
    self.size_pool = 5
    for filter_size in filter_sizes:
      self.conv_layers.append(
          nn.Conv1d(
              in_channels=hidden_size,
              out_channels=hidden_size,
              kernel_size=filter_size,
          )
      )

    self.conv_layers_2 = nn.ModuleList()
    for _ in range(2):
      self.conv_layers_2.append(
          nn.Conv1d(
              in_channels=hidden_size,
              out_channels=hidden_size,
              kernel_size=self.size_pool,
              padding='same'
          )
      )
    self.linear_1 = FCLayer(6144, 768, use_activation='None')
    self.linear_2 = FCLayer(768, 64, use_activation='None')
    self.drop = nn.Dropout(0.2)
    self.debug = debug
    self.pool = nn.MaxPool1d(self.size_pool, stride=self.size_pool)
  def forward(self, sent):
    convs = []
    drop_rate = 0.2
    final_hid = 32

    new_sent = sent.transpose(1,2) #Conv1d input
    cnt = 1
    for conv_layer in self.conv_layers:
      cnt+=1
      l_conv = conv_layer(new_sent)
      # l_conv = l_conv.transpose(1,2).to(device)
      batch_feature = l_conv.size(1)
      if self.debug:
        print('After conv with filter size {0}'.format(cnt),l_conv.size(), batch_feature)
      # l_conv = self.batch_norm(l_conv)
      l_conv = nn.ReLU()(l_conv)
      # l_pool = MaxPooling1D(pool_size=max_len-filter_size+1)(l_conv)
      l_pool = self.pool(l_conv)
      if self.debug:
        print('After pooling', l_pool.size())
      convs.append(l_pool)
      #merge.append(Flatten()(l_pool))

    l2_pool = torch.cat(convs, dim=2)
    if self.debug:
      print('After cat', l2_pool.size())
    # l2_pool = l2_pool.transpose(1,2)
    # l2_pool = BatchNormalization()(l2_pool)
    for conv_layer in self.conv_layers_2:
      origin  = l2_pool
      l2_conv = conv_layer(origin)
      # l2_pool = l2_pool.transpose(1,2)
      batch_feature = l2_conv.size(1)
      # l2_conv = self.batch_norm(l2_conv)
      l2_conv = nn.ReLU()(l2_conv)
      if self.debug:
        print(origin.shape, l2_conv.shape)
      # l2_conv = Add()([Lambda(lambda x: x[0]*x[1])([origin,0.1]), l2_conv])
      l2_conv = l2_conv+origin
      l2_pool = self.pool(l2_conv)
      if self.debug:
        print('After pool', l2_pool.size())
    if self.debug:
      print('After second cnn', l2_pool.size())
    text = l2_pool.view(l2_pool.size(0),-1)
    if self.debug:
      print(text.size())
    #append to merge
    
    text_append = self.linear_1(text)
    batch_feature = text_append.size(1)
    # text_append = self.batch_norm(text_append)
    text_append = self.drop(text_append)
    text_append = nn.ReLU()(text_append)
    text_append = self.linear_2(text_append)
    batch_feature = text_append.size(1)
    # text_append = nn.BatchNorm1d(batch_feature)(text_append)
    text_append = nn.ReLU()(text_append)
    return text_append


In [ ]:
import math

class CustomLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz, droprate=0.1):
        super().__init__()
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.W = nn.Parameter(torch.Tensor(input_sz, hidden_sz * 4))
        self.U = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bias = nn.Parameter(torch.Tensor(hidden_sz * 4))
        self.init_weights()
        self.drop = nn.Dropout(droprate)
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
         
    def forward(self, x, 
                init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = (torch.zeros(bs, self.hidden_size).to(x.device), 
                        torch.zeros(bs, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states
         
        HS = self.hidden_size
        for t in range(seq_sz):
            x_t = x[:, t, :]
            # batch the computations into a single matrix multiplication
            gates = x_t @ self.W + h_t @ self.U + self.bias
            i_t, f_t, g_t, o_t = (
                torch.sigmoid(gates[:, :HS]), # input
                torch.sigmoid(gates[:, HS:HS*2]), # forget
                torch.tanh(gates[:, HS*2:HS*3]),
                torch.sigmoid(gates[:, HS*3:]), # output
            )
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            h_t = self.drop(h_t)
            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)


In [ ]:
class Model(BertPreTrainedModel):
      def __init__(self, config, args):
        super(Model, self).__init__(config)
        self.bert = BertModel(config=config)  # Load pretrained bert
        # for param in self.bert.parameters():
        #   param.requires_grad = False

        self.num_labels = config.num_labels
        self.num_domain = args.num_domains
        self.out_layer = FCLayer(2*args.lstm_hid, args.num_labels, use_activation='None', dropout_rate=0.2)
        self.fc_layer = FCLayer(config.hidden_size*2, config.hidden_size, use_activation='relu', dropout_rate=0.2)
        # self.cnn_extract = CNN_Feature(config.hidden_size)
        # self.drop = nn.Dropout(args.dropout_rate)
        self.lamb = args.lamb
        self.grl = GRL()
        self.domain_layer = FCLayer(2*args.lstm_hid, args.num_domains, use_activation='None', dropout_rate=0.2)
        

        self.lstm_tar_fw = CustomLSTM(config.hidden_size, args.lstm_hid, droprate = args.dropout_rate)
        self.lstm_tar_bw = CustomLSTM(config.hidden_size, args.lstm_hid, droprate = args.dropout_rate)
        self.lstm_sent_fw = CustomLSTM(config.hidden_size, args.lstm_hid, droprate = args.dropout_rate)
        self.lstm_sent_bw = CustomLSTM(config.hidden_size, args.lstm_hid, droprate = args.dropout_rate)
        self.attention_tar = FCLayer(2*args.lstm_hid, 4*args.lstm_hid, use_activation='None')
        self.attention_sent = FCLayer(2*args.lstm_hid, 4*args.lstm_hid, use_activation='None')
        self.attention_layer = FCLayer(4*args.lstm_hid, 1, use_activation='None', set_bias=False)
        self.max_len_sent = 500
        self.hid = args.lstm_hid
      def forward(self, ids_1, masks_1, token_types_1, ids_2, masks_2, token_types_2, labels, dom_labels):
        #BERT feature extractor
        outputs = self.bert(input_ids=ids_1, attention_mask=masks_1, token_type_ids=token_types_1) # sequence_output, pooled_output, (hidden_states), (attentions)
        pooled_output_1 = outputs[2][-1]
        # pooled_output_1 = torch.cat((outputs[2][-1],outputs[2][-2], outputs[2][-3],outputs[2][-4]),2)
        # pooled_output_1 = self.fc_layer(pooled_output_1)
        # pooled_output_1 = torch.cat((outputs[2][-1][:,0,:],outputs[2][-2][:,0,:],outputs[2][-3][:,0,:],outputs[2][-4][:,0,:]),1) # [CLS] batch x 768
        outputs = self.bert(input_ids=ids_2, attention_mask=masks_2, token_type_ids=token_types_2) # sequence_output, pooled_output, (hidden_states), (attentions)
        pooled_output_2 = outputs[2][-1]
        # pooled_output_2 = torch.cat((outputs[2][-1],outputs[2][-2],outputs[2][-3],outputs[2][-4]),2)
        # print(pooled_output_2.size(), pooled_output_2.is_cuda)
        # pooled_output_2 = torch.cat((outputs[2][-1][:,0,:],outputs[2][-2][:,0,:],outputs[2][-3][:,0,:],outputs[2][-4][:,0,:]),1) # [CLS] batch x 768
        # pooled_output_1 = self.cnn_extract(pooled_output_1)
        # pooled_output_2 = self.cnn_extract(pooled_output_2)
        # print(torch.cat((pooled_output_1, pooled_output_2), dim=1).size())

        ##BiCond
        ###FORWARD
        #Target
        fw_tar_outputs , fw_tar_lst_state = self.lstm_tar_fw(pooled_output_1)
        # fw_tar_outputs = self.dropout(fw_tar_outputs)
        #Sentence
        fw_sent_outputs, fw_last = self.lstm_sent_fw(pooled_output_2, fw_tar_lst_state)
        # fw_sent_outputs = self.dropout(fw_sent_outputs)
        # print("forward target outputs", fw_tar_outputs.size())
        # print("forward sent outputs", fw_sent_outputs.size())

        ###BACKWARD
        #Target - start from last -> flip
        bw_tar_outputs , bw_tar_lst_state = self.lstm_tar_bw(pooled_output_1.flip(dims=[1]))  
        # bw_tar_outputs = self.dropout(bw_tar_outputs)
        #Sentence - start from last -> flip
        bw_sent_outputs, bw_last = self.lstm_sent_bw(pooled_output_2.flip(dims=[1]), bw_tar_lst_state)
        # bw_sent_outputs = self.dropout(bw_sent_outputs)
        # att_out = torch.cat((fw_last[0],bw_last[0]), dim=1)

        concat_sent = torch.cat((fw_sent_outputs,bw_sent_outputs.flip(dims=[1])), dim=2)     #batch x seq x 2*hid
        
        # concat_sent = self.dropout(concat_sent)
        b_size = concat_sent.size(0)
        seq = concat_sent.size(1)
        hid_dim = concat_sent.size(2)        
        concat_sent_1 = concat_sent.view(-1, hid_dim)
        concat_sent_1 = self.attention_sent(concat_sent_1)
        concat_sent_1 = concat_sent_1.view(b_size, seq, -1)
        concat_target = torch.cat((fw_tar_outputs[:,-1],bw_tar_outputs[:,-1]), dim=1)   #batch x 2*hid
        # concat_target = self.dropout(concat_target)  
        concat_target = self.attention_tar(concat_target)
        att = concat_target.unsqueeze(1).repeat(1, 1, self.max_len_sent).view(b_size, seq, -1) + concat_sent_1 #batch x seq x 4*hid
        #ATTENTION
        ##input: batch x seq x 4*hid -> reshape -> (batch x seq) x 4*hid -> Linear(tanh) -> (batch x seq) x 4*hid -> Linear(none) -> (batch x seq) x 1 
        #->reshape -> batch x seq -> softmax -> batch x seq -> batch x 1 x seq
        b_size = att.size(0)
        att = att.view(-1, 4*self.hid)
        att = nn.Tanh()(att)
        att = self.attention_layer(att)
        att = att.view(b_size, -1).masked_fill_(masks_2==0, -float('inf'))
        sm = nn.Softmax(dim=1)
        att = sm(att).unsqueeze(1)

        #attention output: batch x 1 x seq
        #seq input: batch x seq x 2*hid -> bmm -> batch x 1 x 2*hid -> batch x 2*hid
        att_out = torch.bmm(att, concat_sent)
        # print("att_out", att_out.size())
        att_out = att_out.squeeze()

        #CLASSIFICATION
        # final = torch.cat((pooled_output_1, pooled_output_2), dim=1)
        final = att_out
        # final = self.fc_layer(final)
        preds = self.out_layer(final)
        # preds = nn.LogSoftmax(dim=1)(preds)
        preds = nn.Sigmoid()(preds)
        #Calculate losses
        loss_fct = nn.BCELoss()
        l_rel = loss_fct(preds.squeeze(), labels.float().view(-1))

        ##GRL
        grl = self.grl(final)
        loss_fct = nn.NLLLoss(reduction='mean')
        domain = nn.LogSoftmax(dim=1)(self.domain_layer(grl))
        l_domain = loss_fct(domain.view(-1, self.num_domain), dom_labels.long().view(-1))  
        loss = l_rel + self.lamb*l_domain

        return loss, preds, l_rel, l_domain, domain, outputs[-1]  #loss, logits, loss_stance, attention

In [ ]:
border = len(df_train[df_train["relevant"] == 1]) / len(df_train["relevant"])
border

0.10096735187424426

In [ ]:
question_ids = df_val['question_id'].unique()
question_ids

array(['q-461', 'q-32', 'q-229', 'q-370', 'q-150', 'q-253', 'q-164',
       'q-275', 'q-214', 'q-166', 'q-448', 'q-350', 'q-366', 'q-153',
       'q-349', 'q-502', 'q-243', 'q-189', 'q-53', 'q-581', 'q-382',
       'q-420', 'q-400', 'q-315', 'q-207', 'q-24', 'q-157', 'q-197',
       'q-86', 'q-111', 'q-190', 'q-322', 'q-248', 'q-241', 'q-124',
       'q-7', 'q-493', 'q-154', 'q-356', 'q-392', 'q-51', 'q-109',
       'q-199', 'q-130', 'q-445', 'q-140', 'q-363', 'q-545', 'q-100',
       'q-42', 'q-531', 'q-294', 'q-251', 'q-449', 'q-17', 'q-413',
       'q-145', 'q-480', 'q-418', 'q-313', 'q-569', 'q-288', 'q-458',
       'q-179', 'q-510', 'q-260', 'q-467', 'q-202', 'q-269', 'q-84',
       'q-592', 'q-266', 'q-411', 'q-552', 'q-83', 'q-169', 'q-177',
       'q-474', 'q-90', 'q-404', 'q-589', 'q-310', 'q-561'], dtype=object)

In [ ]:
from sklearn.metrics import fbeta_score

def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    # print(preds)
    # print(labels)
    return acc_and_f1(preds, labels)


def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels, average="macro"):
    predict = np.where(preds < border, 0, 1)

    acc = simple_accuracy(predict, labels)
    report = classification_report(labels, predict, target_names=['none', 'rel'], output_dict=True)

    f2_macro = 0  
    for id in question_ids:
      indexs = df_val[df_val['question_id']==id].index.values
      fbeta_pred = predict[indexs]
      fbeta_label = labels[indexs]
#       print("ID ",id)
#       print("indexs ",indexs)
#       print("pred",fbeta_pred)
#       print("label",fbeta_label)
      f2_macro += fbeta_score(fbeta_label, fbeta_pred, beta=2) / len(question_ids)

    f1_none = report['none']['f1-score']
    f1_favor = report['rel']['f1-score']
    p_none = report['none']['precision']
    p_favor = report['rel']['precision']
    r_none = report['none']['recall']
    r_favor = report['rel']['recall']

    return {
        "acc": acc,
        "f2_macro": f2_macro,
        "p_none": p_none,
        "p_rel": p_favor,
        "r_rel": r_favor,
        "r_none": r_none,
    }

def init_logger():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)


In [ ]:
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [ ]:
logger = logging.getLogger(__name__)

class Trainer(object):
    def __init__(self, args, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.epochs_stop = args.early_stop
        self.num_labels = args.num_labels

        self.config = BertConfig.from_pretrained(
            args.model_name_or_path,
            num_labels=self.num_labels,
            finetuning_task='VNLaw-BERT',
            output_hidden_states=True,
            output_attentions=True,
        )
        self.model = Model.from_pretrained(args.model_name_or_path, config=self.config, args=args)
        # self.model.resize_token_embeddings(len(tokenizer)) 

        # GPU or CPU
        self.device = 'cuda' if torch.cuda.is_available() and not args.no_cuda else "cpu"
        self.model.to(self.device)

    def train(self):
        train_dataloader = DataLoader(
            self.train_dataset,
            shuffle=True,
            batch_size=self.args.train_batch_size,
            drop_last=True,
        )

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = (
                self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
            )
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": args.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(
            optimizer_grouped_parameters,
            lr=self.args.learning_rate,
            eps=self.args.adam_epsilon,
        )
        optimizer = AdamW(self.model.parameters(), lr=self.args.learning_rate)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.args.warmup_steps,
            num_training_steps=t_total,
        )


        # Train!
        self.args.logging_steps = t_total // self.args.num_train_epochs
        self.args.save_steps    = t_total // self.args.num_train_epochs
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(self.train_dataset))
        logger.info("  Num Epochs = %d", self.args.num_train_epochs)
        logger.info("  Total train batch size = %d", self.args.train_batch_size)
        logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
        logger.info("  Total optimization steps = %d", t_total)
        logger.info("  Logging steps = %d", self.args.logging_steps)
        logger.info("  Save steps = %d", self.args.save_steps)

        global_step = 0
        tr_loss = 0.0
        tr_loss_rel = 0.0
        tr_loss_domain = 0.0
        max_val_score = -1
        early_stop = False
        epochs_no_improve = 0 

        # train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch") 
        for epoch_cnt in range(self.args.num_train_epochs):
            start = time.time()
            self.model.train()

            # epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(train_dataloader):
                optimizer.zero_grad()

                batch = tuple(t.unsqueeze(0).to(self.device) if t.dim()==1 else t.to(self.device) for t in batch)  # GPU or CPU
                inputs = {
                    "ids_1": batch[0],
                    "masks_1": batch[1],
                    "token_types_1": batch[2],
                    "ids_2": batch[3],
                    "masks_2": batch[4],
                    "token_types_2": batch[5],
                    "labels": batch[6],
                    "dom_labels": batch[7],
                }
                outputs = self.model(**inputs)
                loss = outputs[0]

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                tr_loss_rel += outputs[2].item()
                tr_loss_domain += outputs[3].item()
                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    # torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

                    optimizer.step()
                    # scheduler.step()  # Update learning rate schedule

                    if step % 100 == 0 or step == (len(train_dataloader) - 1):
                        logger.info(
                            f"Epoch: [{epoch_cnt + 1}][{step}/{len(train_dataloader)}] "
                            f"Elapsed {timeSince(start, float(step + 1) / len(train_dataloader)):s} "
                            f"Loss: {tr_loss/(global_step+1):.4f} "
                            f"Loss_rel: {tr_loss_rel/(global_step+1):.4f} "
                            f"Loss_domain: {tr_loss_domain/(global_step+1):.4f} "
                        )
                    global_step += 1

                    if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                        res = self.evaluate("dev")
                        logger.info("Training total loss = %.4f", tr_loss/global_step)  
                        if res['f2_macro']  > max_val_score:
                          max_val_score = res['f2_macro']
                          final = res
                          epochs_no_improve = 0
                          self.save_model()
                        else:
                          epochs_no_improve += 1
                        
                        if epochs_no_improve == self.epochs_stop:
                          early_stop = True
                          logger.info(" Early Stopping!!!!!!")
                          logger.info("***** Final results *****")
                          for key in sorted(final.keys()):
                              logger.info("  {} = {:.4f}".format(key, final[key]))
                          break

                    # if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                    #     self.save_model()

                if 0 < self.args.max_steps < global_step or early_stop==True:
                    epoch_iterator.close()
                    break
            
            if 0 < self.args.max_steps < global_step or early_stop==True:
                train_iterator.close()
                break

        return global_step, tr_loss / global_step

    def evaluate(self, mode, out_pred=False):
        # We use test dataset because semeval doesn't have dev dataset
        if mode == "test":
            dataset = self.test_dataset
        elif mode == "dev":
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation on %s dataset *****", mode)
        logger.info("  Num examples = %d", len(dataset))
        logger.info("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        eval_loss_rel = 0.0
        eval_loss_domain = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None

        self.model.eval()

        for batch in eval_dataloader:
            start = time.time()
            batch = tuple(t.unsqueeze(0).to(self.device) if t.dim()==1 else t.to(self.device) for t in batch)  
            with torch.no_grad():
                inputs = {
                    "ids_1": batch[0],
                    "masks_1": batch[1],
                    "token_types_1": batch[2],
                    "ids_2": batch[3],
                    "masks_2": batch[4],
                    "token_types_2": batch[5],
                    "labels": batch[6],
                    "dom_labels": batch[7],
                }
                outputs = self.model(**inputs)
                logits, tmp_eval_loss = outputs[1],outputs[0]
                # print(tmp_eval_loss)
                eval_loss += tmp_eval_loss.item()
                eval_loss_rel += outputs[2].item()
                eval_loss_domain += outputs[3].item()
            if nb_eval_steps % 100 == 0 or nb_eval_steps == (len(eval_dataloader) - 1):
                logger.info(
                            f"VAL: [{nb_eval_steps}/{len(eval_dataloader)}] "
                            f"Elapsed {timeSince(start, float(nb_eval_steps + 1) / len(eval_dataloader)):s} "
                            f"Loss: {eval_loss/(nb_eval_steps+1):.4f} "
                            f"Loss_rel: {eval_loss_rel/(nb_eval_steps+1):.4f} "
                            f"Loss_domain: {eval_loss_domain/(nb_eval_steps+1):.4f} "
                        )
            nb_eval_steps += 1

            if preds is None:
                preds = logits.view(-1).detach().cpu().numpy()
                out_label_ids = inputs["labels"].view(-1).detach().cpu().numpy()
                # att_weights = outputs[-1].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.view(-1).detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].view(-1).detach().cpu().numpy(), axis=0)
                # att_weights = np.append(att_weights, outputs[-1].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        results = {"loss": eval_loss}
        # print(preds)
        # preds = np.argmax(preds, axis=1)
        # write_prediction(self.args, os.path.join(self.args.eval_dir, "proposed_answers.txt"), preds)

        result = compute_metrics(preds, out_label_ids)
        results.update(result)

        logger.info("***** Eval results *****")
        for key in sorted(results.keys()):
            logger.info("  {} = {:.4f}".format(key, results[key]))

        if out_pred == True:
          return preds
        return results

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)
        model_to_save = self.model.module if hasattr(self.model, "module") else self.model
        model_to_save.save_pretrained(self.args.model_dir)
        # model_to_save = self.model
        # torch.save(model_to_save, os.path.join(self.args.model_dir, "model.pt"))

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.args.model_dir, "training_args.bin"))
        logger.info("Saving model checkpoint to %s", self.args.model_dir)

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.args.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        self.args = torch.load(os.path.join(self.args.model_dir, "training_args.bin"))
        self.config = BertConfig.from_pretrained(self.args.model_dir)

        self.model = Model.from_pretrained(self.args.model_dir, config=self.config, args=self.args)
        # self.model = torch.load(os.path.join(self.args.model_dir, "model.pt"))
        self.model.to(self.device)
        logger.info("***** Model Loaded *****")


In [ ]:
def main(args, train_data, valid_data):
    init_logger()
    set_seed(args)

    trainer = Trainer(args, train_dataset=train_data, dev_dataset=val_data)
    if args.do_train:
        trainer.train()

    if args.do_eval:
        trainer.load_model()
        trainer.evaluate("dev")
        # trainer.evaluate("test")


In [ ]:
class arg():
  def __init__(self):
    self.model_dir = '/content/drive/MyDrive/vn_law/BERT_task1_1'
    self.model_name_or_path = MODEL
    self.do_train = True
    self.do_eval = True
    self.no_cuda = False
    # self.logging_steps = 13         #Log every X updates steps
    # self.save_steps = 13            #Save checkpoint every X updates steps
    self.warmup_steps = 0
    self.max_grad_norm = 1.0
    self.adam_epsilon = 1e-8
    self.weight_decay = 0.01
    self.dropout_rate = 0.2
    self.max_steps = -1              #total number of training steps to perform
    self.gradient_accumulation_steps = 1     #Number of updates steps to accumulate before performing a backward/update pass
    self.num_train_epochs = 10
    self.learning_rate = 2e-5
    self.early_stop = 35
    self.train_batch_size = 8
    self.eval_batch_size = 8
    self.seed = 2020
    self.num_labels = 1
    self.lamb = 0.03
    self.lstm_hid = 500
    self.num_domains = 16

In [ ]:
args = arg()

In [ ]:
main(args, train_data, val_data)

You are using a model of type ibert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/checkpoint-810000 were not used when initializing Model: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

KeyboardInterrupt: ignored

In [ ]:
trainer = Trainer(args, train_dataset=train_data, dev_dataset=val_data)
trainer.load_model()

You are using a model of type ibert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/checkpoint-810000 were not used when initializing Model: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

In [ ]:
preds = trainer.evaluate("dev", out_pred=True)

08/24/2021 10:00:55 - INFO - __main__ -   ***** Running evaluation on dev dataset *****
08/24/2021 10:00:55 - INFO - __main__ -     Num examples = 832
08/24/2021 10:00:55 - INFO - __main__ -     Batch size = 8
08/24/2021 10:00:56 - INFO - __main__ -   VAL: [0/104] Elapsed 0m 0s (remain 0m 54s) Loss: 2.3102 Loss_rel: 2.2316 Loss_domain: 2.6190 
08/24/2021 10:01:40 - INFO - __main__ -   VAL: [100/104] Elapsed 0m 0s (remain 0m 0s) Loss: 0.3969 Loss_rel: 0.3090 Loss_domain: 2.9300 
08/24/2021 10:01:42 - INFO - __main__ -   VAL: [103/104] Elapsed 0m 0s (remain 0m 0s) Loss: 0.3957 Loss_rel: 0.3075 Loss_domain: 2.9413 
08/24/2021 10:01:42 - INFO - __main__ -   ***** Eval results *****
08/24/2021 10:01:42 - INFO - __main__ -     acc = 0.8606
08/24/2021 10:01:42 - INFO - __main__ -     f2_macro = 0.6380
08/24/2021 10:01:42 - INFO - __main__ -     loss = 0.3957
08/24/2021 10:01:42 - INFO - __main__ -     p_none = 0.9660
08/24/2021 10:01:42 - INFO - __main__ -     p_rel = 0.4000
08/24/2021 10:01:

In [ ]:
df_val

,Unnamed: 0,question_id,text,law_id,article_id,article_text,relevant,matched_clauses,bert_model
0,0,q-461,Mức thù lao luật sư tham gia tố tụng trong vụ ...,123/2013/NĐ-CP,18,Mức trần thù lao luật sư tham gia tố tụng tron...,1,Mức trần thù lao luật sư tham gia tố tụng tron...,0.291835
1,1,q-32,"Nhà giáo làm nhiệm vụ giảng dạy, giáo dục tron...",43/2019/QH14,66,"Vị trí, vai trò của nhà giáo\n\n1. Nhà giáo là...",1,"Vị trí, vai trò của nhà giáo 1. Nhà giáo làm n...",0.230130
2,2,q-229,Người đăng ký hộ tịch trái phép cho 2 người tr...,100/2015/QH13,336,Tội đăng ký hộ tịch trái pháp luật\n\n1. Người...,1,Tội đăng ký hộ tịch trái pháp luật 3. Người ph...,0.661736
3,3,q-370,"Ủy ban nhân dân tỉnh, thành phố trực thuộc Tru...",45/2013/QH13,30,Bản đồ hành chính\n\n1. Bản đồ hành chính của ...,1,Bản đồ hành chính 2. Việc lập bản đồ hành chín...,0.948693
4,4,q-150,Người chưa thành niên chưa đủ 13 tuổi chỉ được...,45/2019/QH14,145,Sử dụng người chưa đủ 15 tuổi làm việc\n\n1. K...,1,Sử dụng người chưa đủ 15 tuổi làm việc 3. Ngườ...,0.001639
...,...,...,...,...,...,...,...,...,...
827,827,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,21/2010/TT-BTP,6,Thời gian tập sự hành nghề luật sư\n\n1. Thời ...,0,Thời gian tập sự hành nghề luật sư 4. Người tậ...,0.002702
828,828,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,19/2013/TT-BTP,6,Thời gian tập sự hành nghề luật sư\n\n1. Thời ...,0,Thời gian tập sự hành nghề luật sư 4. Người tậ...,0.353408
829,829,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,11/2017/QH14,20,Tập sự trợ giúp pháp lý\n\n1. Viên chức của Tr...,0,Tập sự trợ giúp pháp lý 1. Viên chức của Trung...,0.001045
830,830,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,20/2012/QH13,14,“ Tập sự hành nghề luật sư\n\n1. Người có Giấy...,0,“ Tập sự hành nghề luật sư 1. Người có Giấy ch...,0.001319


In [ ]:
df_val['bert_model'] = preds
df_val

,Unnamed: 0,question_id,text,law_id,article_id,article_text,relevant,matched_clauses,bert_model
0,0,q-461,Mức thù lao luật sư tham gia tố tụng trong vụ ...,123/2013/NĐ-CP,18,Mức trần thù lao luật sư tham gia tố tụng tron...,1,Mức trần thù lao luật sư tham gia tố tụng tron...,0.291835
1,1,q-32,"Nhà giáo làm nhiệm vụ giảng dạy, giáo dục tron...",43/2019/QH14,66,"Vị trí, vai trò của nhà giáo\n\n1. Nhà giáo là...",1,"Vị trí, vai trò của nhà giáo 1. Nhà giáo làm n...",0.230130
2,2,q-229,Người đăng ký hộ tịch trái phép cho 2 người tr...,100/2015/QH13,336,Tội đăng ký hộ tịch trái pháp luật\n\n1. Người...,1,Tội đăng ký hộ tịch trái pháp luật 3. Người ph...,0.661736
3,3,q-370,"Ủy ban nhân dân tỉnh, thành phố trực thuộc Tru...",45/2013/QH13,30,Bản đồ hành chính\n\n1. Bản đồ hành chính của ...,1,Bản đồ hành chính 2. Việc lập bản đồ hành chín...,0.948693
4,4,q-150,Người chưa thành niên chưa đủ 13 tuổi chỉ được...,45/2019/QH14,145,Sử dụng người chưa đủ 15 tuổi làm việc\n\n1. K...,1,Sử dụng người chưa đủ 15 tuổi làm việc 3. Ngườ...,0.001639
...,...,...,...,...,...,...,...,...,...
827,827,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,21/2010/TT-BTP,6,Thời gian tập sự hành nghề luật sư\n\n1. Thời ...,0,Thời gian tập sự hành nghề luật sư 4. Người tậ...,0.002702
828,828,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,19/2013/TT-BTP,6,Thời gian tập sự hành nghề luật sư\n\n1. Thời ...,0,Thời gian tập sự hành nghề luật sư 4. Người tậ...,0.353408
829,829,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,11/2017/QH14,20,Tập sự trợ giúp pháp lý\n\n1. Viên chức của Tr...,0,Tập sự trợ giúp pháp lý 1. Viên chức của Trung...,0.001045
830,830,q-561,Mỗi luật sư hướng dẫn chỉ được hướng dẫn không...,20/2012/QH13,14,“ Tập sự hành nghề luật sư\n\n1. Người có Giấy...,0,“ Tập sự hành nghề luật sư 1. Người có Giấy ch...,0.001319


In [ ]:
from sklearn.preprocessing import MinMaxScaler


for q in df_test['question_id'].unique():
    scores = df_val[df_val['question_id']==q]
    bm25_scores = scores['bm25_score'].values
    model_scores = scores['bert_model'].values
    bm25_scores = bm25_scores.reshape(-1,1)
    # print(bm25_scores)
    scaler = MinMaxScaler()
    bm25_scores = scaler.fit_transform(bm25_scores)
    # print(bm25_scores)
    threshold = 0.7
    final = threshold*bm25_scores + (1-threshold)*model_scores
    print(final.argmax())
    break

KeyError: ignored

# Interfere

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/vn_law/outputs/bm25.csv')
print(df_test.info())
display(df_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401104 entries, 0 to 401103
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   question_id   401104 non-null  object 
 1   text          401104 non-null  object 
 2   law_id        401104 non-null  object 
 3   article_id    401104 non-null  object 
 4   article_text  401104 non-null  object 
 5   relevant      401104 non-null  int64  
 6   bm25_score    401104 non-null  float64
 7   bm25_index    401104 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 24.5+ MB
None


,question_id,text,law_id,article_id,article_text,relevant,bm25_score,bm25_index
0,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,8,Chương trình giáo dục\n\n1. Chương trình giáo ...,1,98.957036,0
1,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,31,Chương trình giáo dục phổ thông\n\n1. Chương t...,0,75.948595,1
2,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,72,Trình độ chuẩn được đào tạo của nhà giáo\n\n1....,0,75.287686,2
3,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,34,"Xác nhận hoàn thành chương trình tiểu học, tru...",0,75.077637,3
4,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,25,Chương trình giáo dục mầm non\n\n1. Chương trì...,0,73.555320,4
...,...,...,...,...,...,...,...,...
401099,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,45/2019/QH14,17,Hành vi người sử dụng lao động không được làm ...,0,0.000000,2274
401100,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,45/2019/QH14,16,Nghĩa vụ cung cấp thông tin khi giao kết hợp đ...,0,0.000000,2275
401101,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,45/2019/QH14,15,Nguyên tắc giao kết hợp đồng lao động\n\n1. Tự...,0,0.000000,2276
401102,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,45/2019/QH14,14,Hình thức hợp đồng lao động\n\n1. Hợp đồng lao...,0,0.000000,2277


In [ ]:
import tqdm

top50 = pd.DataFrame()
for q in tqdm.tqdm(df_test['question_id'].unique()):
    scores = df_test[df_test['question_id']==q]
    top50 = pd.concat([top50,scores[:50]])


100%|██████████| 176/176 [00:04<00:00, 43.80it/s]


In [ ]:
top50

,question_id,text,law_id,article_id,article_text,relevant,bm25_score,bm25_index
0,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,8,Chương trình giáo dục\n\n1. Chương trình giáo ...,1,98.957036,0
1,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,31,Chương trình giáo dục phổ thông\n\n1. Chương t...,0,75.948595,1
2,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,72,Trình độ chuẩn được đào tạo của nhà giáo\n\n1....,0,75.287686,2
3,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,34,"Xác nhận hoàn thành chương trình tiểu học, tru...",0,75.077637,3
4,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,25,Chương trình giáo dục mầm non\n\n1. Chương trì...,0,73.555320,4
...,...,...,...,...,...,...,...,...
398870,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,03/VBHN-VPQH,16,"Người được miễn, giảm thời gian tập sự hành ng...",0,22.695020,45
398871,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,20/2012/QH13,16,"“ Người được miễn, giảm thời gian tập sự hành ...",0,22.657265,46
398872,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,19/2013/TT-BTP,30,Xử lý vi phạm đối với thí sinh tham dự kiểm tr...,0,22.427098,47
398873,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,19/2013/TT-BTP,19,"Trách nhiệm của Bộ Tư pháp\n\n1. Kiểm tra, tha...",0,21.956534,48


In [ ]:
import re

test_sent_1  = top50.text.apply(lambda x: re.sub('\n',' ',x)).values
test_sent_2  = top50.article_text.apply(lambda x: re.sub('\n',' ',x)).values
# train_labels = df_train.label.apply(lambda x: 1 if x==True else 0).values
test_labels = top50.relevant.values
test_domains = le.transform(top50.law_id)


In [ ]:
input_ids_1 = []
attention_masks_1 = []
token_type_ids_1 = []
input_ids_2 = []
attention_masks_2 = []
token_type_ids_2 = []

for sent1, sent2 in zip(test_sent_1, test_sent_2):
  encoded_dict = tokenizer.encode_plus( sent1, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH_Q,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_1.append(encoded_dict['input_ids'])
  attention_masks_1.append(encoded_dict['attention_mask'])
  token_type_ids_1.append(encoded_dict['token_type_ids'])

  encoded_dict = tokenizer.encode_plus( sent2, 
                                        add_special_tokens = True, 
                                        max_length = MAX_LENGTH,         
                                        padding = 'max_length',
                                        return_attention_mask = True,   
                                        return_token_type_ids = True,   
                                        truncation = True,
                                  )
  input_ids_2.append(encoded_dict['input_ids'])
  attention_masks_2.append(encoded_dict['attention_mask'])
  token_type_ids_2.append(encoded_dict['token_type_ids'])


test_input_ids_1 = torch.tensor(input_ids_1)
test_attention_masks_1 = torch.tensor(attention_masks_1)
test_token_type_ids_1 = torch.tensor(token_type_ids_1)
test_input_ids_2 = torch.tensor(input_ids_2)
test_attention_masks_2 = torch.tensor(attention_masks_2)
test_token_type_ids_2 = torch.tensor(token_type_ids_2)
test_labels = torch.tensor(test_labels)
test_domains = torch.tensor(test_domains)
test_input_ids_1.size(), test_attention_masks_1.size(), test_token_type_ids_1.size(), test_input_ids_2.size(), test_attention_masks_2.size(), test_token_type_ids_2.size(), test_labels.size()

(torch.Size([8800, 70]),
 torch.Size([8800, 70]),
 torch.Size([8800, 70]),
 torch.Size([8800, 500]),
 torch.Size([8800, 500]),
 torch.Size([8800, 500]),
 torch.Size([8800]))

In [ ]:
test_data = TensorDataset(test_input_ids_1, test_attention_masks_1, test_token_type_ids_1, test_input_ids_2, test_attention_masks_2, test_token_type_ids_2, test_labels, test_domains)


In [ ]:
trainer = Trainer(args, train_dataset=train_data, dev_dataset=test_data)
trainer.load_model()

You are using a model of type ibert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/checkpoint-810000 were not used when initializing Model: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

In [ ]:
preds = trainer.evaluate("dev", out_pred=True)

08/24/2021 09:15:01 - INFO - __main__ -   ***** Running evaluation on dev dataset *****
08/24/2021 09:15:01 - INFO - __main__ -     Num examples = 8800
08/24/2021 09:15:01 - INFO - __main__ -     Batch size = 8
08/24/2021 09:15:01 - INFO - __main__ -   VAL: [0/1100] Elapsed 0m 0s (remain 9m 59s) Loss: 0.8845 Loss_rel: 0.7874 Loss_domain: 3.2393 
08/24/2021 09:15:46 - INFO - __main__ -   VAL: [100/1100] Elapsed 0m 0s (remain 0m 4s) Loss: 0.2788 Loss_rel: 0.1916 Loss_domain: 2.9073 
08/24/2021 09:16:32 - INFO - __main__ -   VAL: [200/1100] Elapsed 0m 0s (remain 0m 2s) Loss: 0.2932 Loss_rel: 0.2035 Loss_domain: 2.9902 
08/24/2021 09:17:19 - INFO - __main__ -   VAL: [300/1100] Elapsed 0m 0s (remain 0m 1s) Loss: 0.2749 Loss_rel: 0.1825 Loss_domain: 3.0779 
08/24/2021 09:18:08 - INFO - __main__ -   VAL: [400/1100] Elapsed 0m 0s (remain 0m 0s) Loss: 0.2794 Loss_rel: 0.1851 Loss_domain: 3.1430 
08/24/2021 09:18:56 - INFO - __main__ -   VAL: [500/1100] Elapsed 0m 0s (remain 0m 0s) Loss: 0.2769 

In [ ]:
top50['bert_model'] = preds
top50

,question_id,text,law_id,article_id,article_text,relevant,bm25_score,bm25_index,bert_model
0,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,8,Chương trình giáo dục\n\n1. Chương trình giáo ...,1,98.957036,0,0.002352
1,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,31,Chương trình giáo dục phổ thông\n\n1. Chương t...,0,75.948595,1,0.001196
2,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,72,Trình độ chuẩn được đào tạo của nhà giáo\n\n1....,0,75.287686,2,0.056909
3,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,34,"Xác nhận hoàn thành chương trình tiểu học, tru...",0,75.077637,3,0.001930
4,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,25,Chương trình giáo dục mầm non\n\n1. Chương trì...,0,73.555320,4,0.001281
...,...,...,...,...,...,...,...,...,...
398870,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,03/VBHN-VPQH,16,"Người được miễn, giảm thời gian tập sự hành ng...",0,22.695020,45,0.010078
398871,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,20/2012/QH13,16,"“ Người được miễn, giảm thời gian tập sự hành ...",0,22.657265,46,0.008798
398872,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,19/2013/TT-BTP,30,Xử lý vi phạm đối với thí sinh tham dự kiểm tr...,0,22.427098,47,0.005407
398873,q-530,Thẩm quyền tổ chức kiểm tra kết quả tập sự hàn...,19/2013/TT-BTP,19,"Trách nhiệm của Bộ Tư pháp\n\n1. Kiểm tra, tha...",0,21.956534,48,0.001055


In [ ]:
from sklearn.preprocessing import MinMaxScaler

final_csv = pd.DataFrame()
final_json = []
id = []
text = []
law_id = []
article_id = []
article_text = []
bm25_score = []
bm25_index = []
bert_model = []
en_score = []
for q in tqdm.tqdm(df_test['question_id'].unique()):
    scores = top50[top50['question_id']==q]
    bm25_scores = scores['bm25_score'].values
    model_scores = scores['bert_model'].values
    bm25_scores = bm25_scores.reshape(-1,1)
    model_scores = model_scores.reshape(-1,1)
    # print(bm25_scores)
    scaler = MinMaxScaler()
    bm25_scores = scaler.fit_transform(bm25_scores)
    model_scores = scaler.fit_transform(model_scores)
    # print(bm25_scores)
    threshold = 0.95
    final = threshold*bm25_scores + (1-threshold)*model_scores
    info = scores.iloc[final.argmax()]

    id.append(info['question_id'])
    text.append(info['text'])
    law_id.append(info['law_id'])
    article_id.append(info['article_id'])
    article_text.append(info['article_text'])
    bm25_score.append(info['bm25_score'])
    bm25_index.append(info['bm25_index'])
    bert_model.append(info['bert_model'])
    en_score.append(final.max())

    ans = {}
    ans['question_id'] = info['question_id']
    ans['relevant_articles'] = []
    for index in range(1):
        rel_article = {}
        rel_article['law_id'] = info['law_id']
        rel_article['article_id'] = info['article_id']
        ans['relevant_articles'].append(rel_article)
    final_json.append(ans)
    # print(final_json)
    # break

final_csv['question_id'] = id 
final_csv['text'] = text 
final_csv['law_id'] = law_id 
final_csv['article_id'] = article_id
final_csv['article_text'] = article_text
final_csv['bm25_score'] =bm25_score
final_csv['bm25_index'] =bm25_index
final_csv['bert_model'] =bert_model
final_csv['en_score'] = en_score


100%|██████████| 176/176 [00:00<00:00, 622.82it/s]


In [ ]:
final_csv

,question_id,text,law_id,article_id,article_text,bm25_score,bm25_index,bert_model,en_score
0,q-8,"Chuẩn kiến thức, kỹ năng, yêu cầu cần đạt về p...",43/2019/QH14,8,Chương trình giáo dục\n\n1. Chương trình giáo ...,98.957036,0,0.002352,0.950062
1,q-113,Ủy ban nhân dân có quyền tuyên bố hợp đồng lao...,45/2019/QH14,50,Thẩm quyền tuyên bố hợp đồng lao động vô hiệu\...,31.295999,0,0.504207,0.976334
2,q-193,Sử dụng bảo vật quốc gia với mục đích trái phé...,100/2015/QH13,177,Tội sử dụng trái phép tài sản\n\n1. Người nào ...,29.865201,0,0.008061,0.950416
3,q-539,Chứng chỉ hành nghề luật sư bị thu hồi khi khô...,65/2006/QH11,18,Thu hồi Chứng chỉ hành nghề luật sư\n\n1. Ngườ...,31.338341,0,0.001478,0.950035
4,q-48,Nơi cư trú của công dân có thể là nơi tạm trú.,68/2020/QH14,11,Nơi cư trú của công dân\n\n1. Nơi cư trú của c...,34.313530,0,0.503403,0.975748
...,...,...,...,...,...,...,...,...,...
171,q-364,Bộ Tài nguyên và Môi trường chịu trách nhiệm t...,45/2013/QH13,23,Trách nhiệm quản lý nhà nước về đất đai\n\n1. ...,53.337361,0,0.451064,0.973028
172,q-477,Báo cáo quá trình tập sự hành nghề luật sư của...,19/2013/TT-BTP,11,Báo cáo quá trình tập sự hành nghề luật sư\n\n...,81.761348,0,0.918821,1.000000
173,q-487,Luật sư hướng dẫn tập sự hành nghề luật sư có ...,19/2013/TT-BTP,12,Điều kiện đối với luật sư hướng dẫn\n\nLuật sư...,57.510731,0,0.946033,1.000000
174,q-274,Tài sản được giao theo phương thức do các bên ...,91/2015/QH13,436,Phương thức giao tài sản\n\n1. Tài sản được gi...,82.338358,0,0.910167,0.997023


In [ ]:
import json
with open('/content/drive/MyDrive/vn_law/outputs/output_bm25_bert_domain.json', 'w', encoding='utf-8') as f:
    json.dump(final_json, f, indent=4, ensure_ascii=False)
final_csv.to_csv('/content/drive/MyDrive/vn_law/outputs/bm25_bert_domain.csv',index=False)

In [ ]:
final_json

[{'question_id': 'q-8',
  'relevant_articles': [{'article_id': '2', 'law_id': '123/2013/NĐ-CP'}]},
 {'question_id': 'q-113',
  'relevant_articles': [{'article_id': '66', 'law_id': '45/2013/QH13'}]},
 {'question_id': 'q-193',
  'relevant_articles': [{'article_id': '293', 'law_id': '100/2015/QH13'}]},
 {'question_id': 'q-539',
  'relevant_articles': [{'article_id': '17', 'law_id': '137/2018/NĐ-CP'}]},
 {'question_id': 'q-48',
  'relevant_articles': [{'article_id': '5', 'law_id': '68/2020/QH14'}]},
 {'question_id': 'q-23',
  'relevant_articles': [{'article_id': '14', 'law_id': '43/2019/QH14'}]},
 {'question_id': 'q-217',
  'relevant_articles': [{'article_id': '127', 'law_id': '45/2019/QH14'}]},
 {'question_id': 'q-183',
  'relevant_articles': [{'article_id': '138', 'law_id': '100/2015/QH13'}]},
 {'question_id': 'q-423',
  'relevant_articles': [{'article_id': '17', 'law_id': '137/2018/NĐ-CP'}]},
 {'question_id': 'q-133',
  'relevant_articles': [{'article_id': '106', 'law_id': '45/2019/QH14

In [ ]:
top50.columns

Index(['question_id', 'text', 'law_id', 'article_id', 'article_text',
       'relevant', 'bm25_score', 'bm25_index', 'bert_model'],
      dtype='object')

In [ ]:
df_val['relevant'].values

In [ ]:
#BERT
08/20/2021 17:24:53 - INFO - __main__ -     acc = 0.7067
08/20/2021 17:24:53 - INFO - __main__ -     f2_macro = 0.5643
08/20/2021 17:24:53 - INFO - __main__ -     loss = 0.2703
08/20/2021 17:24:53 - INFO - __main__ -     p_none = 0.9632
08/20/2021 17:24:53 - INFO - __main__ -     p_rel = 0.2249
08/20/2021 17:24:53 - INFO - __main__ -     r_none = 0.7001
08/20/2021 17:24:53 - INFO - __main__ -     r_rel = 0.7647

#BERT, domain
08/20/2021 17:46:19 - INFO - __main__ -   ***** Eval results *****
08/20/2021 17:46:19 - INFO - __main__ -     acc = 0.7885
08/20/2021 17:46:19 - INFO - __main__ -     f2_macro = 0.6128
08/20/2021 17:46:19 - INFO - __main__ -     loss = 0.2838
08/20/2021 17:46:19 - INFO - __main__ -     p_none = 0.9673
08/20/2021 17:46:19 - INFO - __main__ -     p_rel = 0.2941
08/20/2021 17:46:19 - INFO - __main__ -     r_none = 0.7912
08/20/2021 17:46:19 - INFO - __main__ -     r_rel = 0.7647

#newBERT, domain
08/22/2021 15:36:08 - INFO - __main__ -   ***** Eval results *****
08/22/2021 15:36:08 - INFO - __main__ -     acc = 0.8570
08/22/2021 15:36:08 - INFO - __main__ -     f2_macro = 0.6114
08/22/2021 15:36:08 - INFO - __main__ -     loss = 0.4314
08/22/2021 15:36:08 - INFO - __main__ -     p_none = 0.9631
08/22/2021 15:36:08 - INFO - __main__ -     p_rel = 0.3896
08/22/2021 15:36:08 - INFO - __main__ -     r_none = 0.8742
08/22/2021 15:36:08 - INFO - __main__ -     r_rel = 0.7059
08/22/2021 15:36:08 - INFO - __main__ -   Training total loss = 0.3205

#newBert, domain, lstm
08/22/2021 17:16:16 - INFO - __main__ -     acc = 0.8606
08/22/2021 17:16:16 - INFO - __main__ -     f2_macro = 0.6380
08/22/2021 17:16:16 - INFO - __main__ -     loss = 0.3957
08/22/2021 17:16:16 - INFO - __main__ -     p_none = 0.9660
08/22/2021 17:16:16 - INFO - __main__ -     p_rel = 0.4000
08/22/2021 17:16:16 - INFO - __main__ -     r_none = 0.8755
08/22/2021 17:16:16 - INFO - __main__ -     r_rel = 0.7294


In [ ]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .


Processing /content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2667071 sha256=0c2cd3041e4631d5cc909fb18178d8f8d1823b7b24e9a19d440250b08d2ad60a
  Stored in directory: /tmp/pip-ephem-wheel-cache-q84veem2/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Cloning into 'transformers'...
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
python3 run_mlm.py \
    --model_name_or_path NlpHUST/vibert4news-base-cased \
    --train_file /content/drive/MyDrive/vn_law/law.vi.txt \
    --validation_file /content/drive/MyDrive/vn_law/law.vi.small.txt \
    --do_train \
    --do_eval \
    --line_by_line \
    --validation_split_percentage 5 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 10 \
    --max_seq_length 512 \
    --logging_steps=5000 \
    --save_steps=5000 \
    --eval_steps=5000 \
    --evaluation_strategy steps \
    --output_dir /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm


In [ ]:
!python3 run_mlm.py \
    --model_name_or_path /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/checkpoint-285000 \
    --validation_file /content/drive/MyDrive/vn_law/law.vi.small.txt \
    --do_eval \
    --line_by_line \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --max_seq_length 512 \
    --logging_steps=5000 \
    --save_steps=5000 \
    --eval_steps=5000 \
    --evaluation_strategy steps \
    --output_dir /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm


08/23/2021 11:46:54 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/23/2021 11:46:54 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=5000,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=/conten

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 55 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 264 kB 29.0 MB/s 
     |████████████████████████████████| 1.2 MB 62.9 MB/s 
     |████████████████████████████████| 118 kB 69.3 MB/s 
     |████████████████████████████████| 243 kB 75.9 MB/s 


In [ ]:
!pwd

/content


In [ ]:
!tensorboard dev upload --logdir /content/drive/MyDrive/vn_law/vnlaw-finetune-mlm/runs \


2021-08-23 17:25:08.716482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 17:25:09.140872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 17:25:09.141405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/sdGVTRKwTxeq2DwjYeGzmQ/

[2021-08-23T17:25:09] Started scanning logdir.
[2021-08-23T17:25:27] Total uploaded: 896 scalars, 70 tensors (42